# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.11.4.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [1]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using blogs from PMarca (Marc Andreessen) as our sample data. This data is largely irrelevant as we want to focus on the mechanisms of RAG, which includes out data's shape and quality - but not specifically what the contents of the data are. 


In [3]:
text_loader = TextFileLoader("data/PMarcaBlogs.txt")
documents = text_loader.load_documents()
len(documents)

1

In [4]:
print(documents[0][:100])


The Pmarca Blog Archives
(select posts from 2007-2009)
Marc Andreessen
copyright: Andreessen Horow


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [5]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

373

Let's take a look at some of the documents we've managed to split.

In [6]:
split_documents[0:1]

['\ufeff\nThe Pmarca Blog Archives\n(select posts from 2007-2009)\nMarc Andreessen\ncopyright: Andreessen Horowitz\ncover design: Jessica Hagy\nproduced using: Pressbooks\nContents\nTHE PMARCA GUIDE TO STARTUPS\nPart 1: Why not to do a startup 2\nPart 2: When the VCs say "no" 10\nPart 3: "But I don\'t know any VCs!" 18\nPart 4: The only thing that matters 25\nPart 5: The Moby Dick theory of big companies 33\nPart 6: How much funding is too little? Too much? 41\nPart 7: Why a startup\'s initial business plan doesn\'t\nmatter that much\n49\nTHE PMARCA GUIDE TO HIRING\nPart 8: Hiring, managing, promoting, and Dring\nexecutives\n54\nPart 9: How to hire a professional CEO 68\nHow to hire the best people you\'ve ever worked\nwith\n69\nTHE PMARCA GUIDE TO BIG COMPANIES\nPart 1: Turnaround! 82\nPart 2: Retaining great people 86\nTHE PMARCA GUIDE TO CAREER, PRODUCTIVITY,\nAND SOME OTHER THINGS\nIntroduction 97\nPart 1: Opportunity 99\nPart 2: Skills and education 107\nPart 3: Where to go and wh

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [7]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
2. What technique does OpenAI use to achieve this?

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #2!

### Answers
1. Yes, you can modify the dimension using the `dimensions` parameter in the API.
2. OpenAI uses *Matryoshka Representation Learning (MRL)* to enable dimension reduction.

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [8]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

###Answer
The benefits of using async for embeddings include non-blocking operations allowing other code to run -> e.g. when an I/O operation is blocked, another task can be run within the async io framework instead of blocking that other tasks. Hence, when you're dealing with multiple documents, it allows for more efficient resource usage. In this case, we're making API calls and we don't need to block on an individual call as a result of using async! The allows for greater concurrency and faster run times.

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [9]:
vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)

[('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who has a weak\nproduct management executive. The CEO who used to be in\nsales who has a weak sales executive. The CEO who used to be\nin marketing who has a weak marketing executive.\nI call this the “Michael Eisner Memorial Weak Executive Problem” — aaer the CEO of Disney who had previously been a brilliant TV network executive. When he bought ABC at Disney, it\npromptly fell to fourth place. His response? “If I had an extra\ntwo days a week, I could turn around ABC myself.” Well, guess\nwhat, he didn’t have an extra two days a week.\nA CEO — or a startup founder — oaen has a hard time letting\ngo of the function that brought him to the party. The result: you\nhire someone weak into the executive role for that function so\nthat you can continue to b

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4o-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

### Answer

Yes, you can use the seed parameter (set to the same integer across requests) and temperature=0 to achieve more deterministic outputs, though perfect reproducibility is not guaranteed. You should also monitor the system_fingerprint field to detect backend changes that might affect consistency. Keep all other parameters (prompt, max_tokens, top_p) identical across requests for best results.

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

### Creating and Prompting OpenAI's `gpt-4o-mini`!

Let's tie all these together and use it to prompt `gpt-4o-mini`!

In [10]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [12]:
print(response)

The best way to write a loop in Python often depends on the context of what you're trying to accomplish. However, here are a few tips to write effective loops:

1. **Use the Right Type of Loop**:
   - For iterating over a sequence (like a list, tuple, or string), consider using a `for` loop. This is usually cleaner and more Pythonic than a `while` loop.
   - Use a `while` loop when you need to repeat an action until a certain condition is met.

2. **Keep It Readable**:
   - Use meaningful variable names and maintain a clear structure. This improves code readability.

3. **Avoid Modifying the List While Iterating**:
   - If you need to remove elements from a list within a loop, consider creating a copy of the list or using list comprehensions.

4. **Use `enumerate()`**:
   - When you need both the index and the value from a list, use `enumerate()` to avoid manual indexing.

5. **Consider List Comprehensions**:
   - For simple cases, you can often achieve the same result more concisely w

## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [14]:
RAG_SYSTEM_TEMPLATE = """You are a knowledgeable assistant that answers questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't know"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [15]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [16]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "What is the 'Michael Eisner Memorial Weak Executive Problem'?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")


Response: The 'Michael Eisner Memorial Weak Executive Problem' refers to a situation where a CEO, who has a strong background in a particular area (such as product management, sales, or marketing), hires a weak executive for that same function. This often occurs because the CEO finds it difficult to let go of the function that contributed to their success and thus chooses a less competent individual for the role to maintain their own prominence in that area. The example provided in the context describes Michael Eisner, who was previously a successful TV network executive but, after acquiring ABC for Disney, failed to improve its performance and expressed a belief that he could turn it around if he had more time. This illustrates the issue of hiring a weak executive to allow the CEO to continue feeling like a key player in that domain (Source 1).

Context Count: 3
Similarity Scores: ['Source 1: 0.658', 'Source 2: 0.509', 'Source 3: 0.479']


#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

### Answer
The primary strategy is called "Chain-of-Thought (CoT) Prompting," where you add phrases like "Let's think step by step" to encourage the model to break down problems into intermediate reasoning steps. "Few-Shot Prompting" is another effective technique where you provide 2-5 examples of detailed responses, and the model learns to match that pattern and depth. For the most thoughtful responses, you can combine both methods by showing examples that include step-by-step reasoning.

> NOTE: You can look through ["Accessing GPT-3.5-turbo Like a Developer"](https://colab.research.google.com/drive/1mOzbgf4a2SP5qQj33ZxTz2a01-5eXqk2?usp=sharing) for an answer to this question if you get stuck!

### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database

While these are suggestions, you should feel free to make whatever augmentations you desire! 

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

> NOTE: If you're not sure where to start - ask Cursor (CMD/CTRL+L) to guide you through the changes!

### Activity #1
For Activity #1, I will add metadata support to the vector database.

In [17]:
# Activity #1: Metadata Support Implementation - COMPLETE DEMO
print("🚀 Activity #1: Enhanced RAG with Metadata Support")
print("=" * 60)

# Step 1: Load documents with enhanced metadata
from aimakerspace.text_utils import MetadataEnhancedLoader

print("📁 Loading documents with comprehensive metadata...")
enhanced_loader = MetadataEnhancedLoader("data/PMarcaBlogs.txt")
chunks_with_metadata, chunk_metadata = enhanced_loader.load_and_split()

print(f"✅ Loaded {len(chunks_with_metadata)} chunks with rich metadata")
print(f"✅ Generated {len(chunk_metadata)} metadata records")

# Show metadata capabilities
print(f"\n📊 Metadata Summary:")
summary = enhanced_loader.get_summary()
for key, value in summary.items():
    print(f"   {key}: {value}")

print(f"\n📋 Sample Chunk Metadata (showing first 10 fields):")
if chunk_metadata:
    sample = chunk_metadata[0]
    for key, value in list(sample.items())[:10]:
        print(f"   {key}: {value}")

# Step 2: Build enhanced vector database
print(f"\n🔧 Building enhanced vector database with metadata...")
enhanced_vector_db = VectorDatabase()
enhanced_vector_db = await enhanced_vector_db.abuild_from_list(chunks_with_metadata, chunk_metadata)

stats = enhanced_vector_db.get_stats()
print(f"✅ Enhanced database statistics:")
print(f"   - Total documents: {stats['total_documents']}")
print(f"   - Metadata fields available: {len(stats['metadata_keys'])}")
print(f"   - Key metadata fields: {stats['metadata_keys'][:8]}...")

# Step 3: Demonstrate filtered search capabilities
print(f"\n🔍 DEMO: Advanced Filtered Search Capabilities")

queries_and_filters = [
    {
        "query": "What is the Michael Eisner Memorial Weak Executive Problem?",
        "filters": None,
        "name": "🔍 Regular Search (No Filters)"
    },
    {
        "query": "What is the Michael Eisner Memorial Weak Executive Problem?", 
        "filters": {
            "chunk_index": {"min": 0, "max": 50},
            "word_count": {"min": 50}  # Include reasonable word count
        },
        "name": "📍 Early Document Search (First 50 chunks)"
    },
    {
        "query": "startup funding advice",
        "filters": {
            "word_count": {"min": 150}
        },
        "name": "📝 Substantial Funding Advice (150+ words)"
    },
    {
        "query": "executive hiring",
        "filters": {"word_count": {"min": 100}, "chunk_index": {"max": 100}},
        "name": "🎯 Combined Filters (Substantial + Early chunks)"
    }
]

for i, test_case in enumerate(queries_and_filters, 1):
    print(f"\n{'-' * 50}")
    print(f"{test_case['name']}")
    print(f"Query: '{test_case['query']}'")
    print(f"Filters: {test_case['filters']}")
    print(f"{'-' * 50}")
    
    results = enhanced_vector_db.search_by_text(
        test_case["query"],
        k=3,
        filters=test_case["filters"],
        include_metadata=True
    )
    
    print(f"📊 Found {len(results)} results:")
    for j, (text, score, metadata) in enumerate(results, 1):
        chunk_idx = metadata.get('chunk_index', 'N/A')
        word_count = metadata.get('word_count', 'N/A') 
        source_file = metadata.get('source_source_file', 'N/A')
        print(f"   Result {j}: Chunk #{chunk_idx}, {word_count} words, Score: {score:.3f}")
        print(f"   Preview: {text[:100]}...")
        print()

# Step 4: Enhanced RAG Pipeline Integration
print(f"\n🤖 DEMO: Enhanced RAG Pipeline with Metadata")

class MetadataEnhancedRAGPipeline:
    def __init__(self, llm, vector_db, include_source_info=True):
        self.llm = llm
        self.vector_db = vector_db
        self.include_source_info = include_source_info
        
    def run_pipeline(self, user_query, k=4, filters=None):
        # Search with optional metadata filtering
        results = self.vector_db.search_by_text(
            user_query, k=k, filters=filters, include_metadata=True
        )
        
        # Build enhanced context with metadata
        context_parts = []
        metadata_summary = []
        
        for i, (text, score, metadata) in enumerate(results, 1):
            context_parts.append(f"[Source {i}]: {text}")
            
            if self.include_source_info:
                source_info = []
                if "chunk_index" in metadata:
                    source_info.append(f"Chunk: {metadata['chunk_index']}")
                if "word_count" in metadata:
                    source_info.append(f"Words: {metadata['word_count']}")
                if "source_source_file" in metadata:
                    source_info.append(f"File: {metadata['source_source_file']}")
                
                metadata_summary.append(f"Source {i} Info: {', '.join(source_info)}")
        
        context_text = "\n\n".join(context_parts)
        metadata_text = "\n".join(metadata_summary)
        
        # Create enhanced prompts
        system_message = rag_system_prompt.create_message(
            response_style="detailed",
            response_length="comprehensive"
        )
        
        user_message = rag_user_prompt.create_message(
            user_query=user_query,
            context=f"{context_text}\n\nSource Metadata:\n{metadata_text}",
            context_count=len(results),
            similarity_scores=""
        )
        
        response = self.llm.run([system_message, user_message])
        
        return {
            "response": response,
            "sources_used": len(results),
            "metadata_applied": bool(filters),
            "source_info": metadata_summary
        }

# Test enhanced RAG pipeline
enhanced_rag = MetadataEnhancedRAGPipeline(chat_openai, enhanced_vector_db)

print(f"\n🧪 Testing Enhanced RAG Pipeline:")
result = enhanced_rag.run_pipeline(
    "What is the Michael Eisner Memorial Weak Executive Problem?",
    k=3,
    filters={"word_count": {"min": 100}}  # Only substantial chunks
)

print(f"📝 Enhanced RAG Response:")
print(f"{result['response']}")
print(f"\n📊 Pipeline Metadata:")
print(f"   Sources used: {result['sources_used']}")
print(f"   Metadata filtering applied: {result['metadata_applied']}")
print(f"   Source information: {result['source_info']}")

# Step 5: Comparison with original pipeline
print(f"\n⚖️  COMPARISON: Original vs Enhanced Pipeline")

# Original pipeline result (for comparison)
original_result = rag_pipeline.run_pipeline(
    "What is the Michael Eisner Memorial Weak Executive Problem?",
    k=3
)

print(f"📊 Comparison Summary:")
print(f"   Original pipeline sources: {original_result['context_count']}")  
print(f"   Enhanced pipeline sources: {result['sources_used']}")
print(f"   Enhanced pipeline added: Metadata filtering, source attribution, chunk selection")

print(f"\n🎉 ACTIVITY #1 COMPLETE!")
print(f"✨ Successfully implemented METADATA SUPPORT with:")
print(f"   ✅ Document and chunk metadata tracking")
print(f"   ✅ Advanced filtering capabilities")  
print(f"   ✅ Enhanced search with metadata")
print(f"   ✅ Source attribution in responses")
print(f"   ✅ Backward compatibility maintained")
print(f"   ✅ Production-ready metadata system")

print(f"\n🚀 Your RAG system is now enhanced with professional-grade metadata capabilities!")

🚀 Activity #1: Enhanced RAG with Metadata Support
📁 Loading documents with comprehensive metadata...
✅ Loaded 373 chunks with rich metadata
✅ Generated 373 metadata records

📊 Metadata Summary:
   total_documents: 1
   total_chunks: 373
   total_characters: 297904
   total_words: 52305
   average_chunk_size: 998.1340482573727
   source_files: ['PMarcaBlogs.txt']
   chunk_size_distribution: {'min': 304, 'max': 1000, 'avg': 998.1340482573727}

📋 Sample Chunk Metadata (showing first 10 fields):
   chunk_index: 0
   chunk_start_pos: 0
   chunk_end_pos: 1000
   chunk_size: 1000
   word_count: 181
   line_count: 36
   overlap_size: 0
   is_first_chunk: True
   is_last_chunk: False
   chunk_hash: 8d2eed05

🔧 Building enhanced vector database with metadata...
✅ Enhanced database statistics:
   - Total documents: 373
   - Metadata fields available: 29
   - Key metadata fields: ['word_count', 'source_doc_hash', 'overlap_size', 'is_first_chunk', 'sentiment_label', 'source_text_length', 'chunk_has

In [18]:
# 🔬 SENTIMENT ANALYSIS TESTING - Activity #1 Extension
print("🎯 TESTING: Enhanced Metadata with Sentiment Analysis")
print("=" * 60)

# Step 1: Reload with sentiment analysis
print("📁 Reloading documents with sentiment analysis...")
sentiment_loader = MetadataEnhancedLoader("data/PMarcaBlogs.txt", analyze_sentiment=True)
sentiment_chunks, sentiment_metadata = sentiment_loader.load_and_split()

print(f"✅ Loaded {len(sentiment_chunks)} chunks with sentiment analysis")
print(f"📊 Total metadata fields per chunk: {len(sentiment_metadata[0])}")

# Step 2: Show sentiment-specific metadata fields
print("\n📋 Sentiment Analysis Fields:")
sample_metadata = sentiment_metadata[0]
sentiment_fields = [k for k in sample_metadata.keys() if 'sentiment' in k.lower()]
for field in sentiment_fields:
    print(f"   ✨ {field}: {sample_metadata[field]}")

# Step 3: Sentiment statistics
print("\n📊 Sentiment Distribution Analysis:")
sentiment_labels = [m['sentiment_label'] for m in sentiment_metadata]
sentiment_counts = {}
for label in sentiment_labels:
    sentiment_counts[label] = sentiment_counts.get(label, 0) + 1

for label, count in sentiment_counts.items():
    percentage = (count / len(sentiment_metadata)) * 100
    print(f"   {label.title()}: {count} chunks ({percentage:.1f}%)")

# Step 4: Sentiment score statistics
compound_scores = [m['sentiment_compound'] for m in sentiment_metadata]
print(f"\n📈 Sentiment Score Analysis:")
print(f"   Most Positive: {max(compound_scores):.3f}")
print(f"   Most Negative: {min(compound_scores):.3f}")
print(f"   Average: {sum(compound_scores) / len(compound_scores):.3f}")

# Step 5: Build vector database with sentiment
print(f"\n🔧 Building vector database with sentiment metadata...")
sentiment_vector_db = VectorDatabase()
sentiment_vector_db = await sentiment_vector_db.abuild_from_list(sentiment_chunks, sentiment_metadata)

# Step 6: Test sentiment-based filtering
print(f"\n🔍 DEMO: Sentiment-Based Search Filtering")

sentiment_tests = [
    {
        "query": "startup advice and tips",
        "filters": {"sentiment_label": "positive"},
        "name": "😊 Positive Startup Advice Only"
    },
    {
        "query": "business challenges and problems", 
        "filters": {"sentiment_compound": {"min": -1.0, "max": 0.0}},
        "name": "😐 Negative/Neutral Business Content"
    },
    {
        "query": "entrepreneurship success",
        "filters": {
            "sentiment_compound": {"min": 0.1},  # Positive sentiment
            "word_count": {"min": 150}           # Substantial content
        },
        "name": "🚀 Highly Positive & Substantial Entrepreneurship Content"
    }
]

for test in sentiment_tests:
    print(f"\n{'-' * 50}")
    print(f"{test['name']}")
    print(f"Query: '{test['query']}'")
    print(f"Filters: {test['filters']}")
    print(f"{'-' * 50}")
    
    results = sentiment_vector_db.search_by_text(
        test["query"],
        k=3,
        filters=test["filters"],
        include_metadata=True
    )
    
    print(f"📊 Found {len(results)} results:")
    for j, (text, score, metadata) in enumerate(results, 1):
        sentiment_label = metadata.get('sentiment_label', 'unknown')
        sentiment_score = metadata.get('sentiment_compound', 0)
        word_count = metadata.get('word_count', 0)
        chunk_idx = metadata.get('chunk_index', 'N/A')
        
        print(f"   Result {j}: Chunk #{chunk_idx}, Sentiment: {sentiment_label} ({sentiment_score:.3f})")
        print(f"   Score: {score:.3f}, Words: {word_count}")
        print(f"   Preview: {text[:120]}...")
        print()

# Step 7: Enhanced RAG with sentiment awareness
print(f"\n🤖 DEMO: Sentiment-Aware RAG Pipeline")

enhanced_rag_sentiment = MetadataEnhancedRAGPipeline(chat_openai, sentiment_vector_db)

print(f"\n🧪 Testing Sentiment-Filtered RAG:")
positive_result = enhanced_rag_sentiment.run_pipeline(
    "Give me advice about startup funding",
    k=3,
    filters={"sentiment_label": "positive", "word_count": {"min": 100}}
)

print(f"📝 Positive-Sentiment RAG Response:")
print(f"{positive_result['response']}")
print(f"\n📊 Sentiment Pipeline Stats:")
print(f"   Sources used: {positive_result['sources_used']}")
print(f"   Filtering: Positive sentiment + 100+ words")

print(f"\n🎉 SENTIMENT ANALYSIS ENHANCEMENT COMPLETE!")
print(f"✨ Successfully added sentiment analysis with:")
print(f"   ✅ VADER sentiment scoring (compound, positive, negative, neutral)")
print(f"   ✅ Sentiment label classification (positive/negative/neutral)")
print(f"   ✅ Sentiment-based filtering in search")
print(f"   ✅ Sentiment-aware RAG pipeline")
print(f"   ✅ {len(sentiment_chunks)} chunks analyzed for sentiment")

sentiment_stats = {
    'positive': sentiment_counts.get('positive', 0),
    'negative': sentiment_counts.get('negative', 0), 
    'neutral': sentiment_counts.get('neutral', 0)
}
print(f"   📊 Distribution: {sentiment_stats['positive']} positive, {sentiment_stats['negative']} negative, {sentiment_stats['neutral']} neutral")


🎯 TESTING: Enhanced Metadata with Sentiment Analysis
📁 Reloading documents with sentiment analysis...
✅ Loaded 373 chunks with sentiment analysis
📊 Total metadata fields per chunk: 29

📋 Sentiment Analysis Fields:
   ✨ sentiment_compound: 0.9265
   ✨ sentiment_positive: 0.101
   ✨ sentiment_negative: 0.044
   ✨ sentiment_neutral: 0.855
   ✨ sentiment_label: positive

📊 Sentiment Distribution Analysis:
   Positive: 310 chunks (83.1%)
   Negative: 60 chunks (16.1%)
   Neutral: 3 chunks (0.8%)

📈 Sentiment Score Analysis:
   Most Positive: 0.997
   Most Negative: -0.985
   Average: 0.601

🔧 Building vector database with sentiment metadata...

🔍 DEMO: Sentiment-Based Search Filtering

--------------------------------------------------
😊 Positive Startup Advice Only
Query: 'startup advice and tips'
Filters: {'sentiment_label': 'positive'}
--------------------------------------------------
📊 Found 3 results:
   Result 1: Chunk #241, Sentiment: positive (0.971)
   Score: 0.498, Words: 177
   